## Tables and Data Source setup

In [1]:
using ODBC
using DataFrames
using DSWB
using Formatting
using URIParser
using JSON

dsn = "dswb-natgeo" # Redshift esetTable(tableRt, tableType = "RESOURCE_TABLE")ndpoint
table = "beacons_4744" # beacon table name
tableRt = "beacons_4744_rt"

# Connect to Beacon Data
setRedshiftEndpoint(dsn)
setTable(table)
setTable(tableRt, tableType = "RESOURCE_TABLE")
;

Redshift endpoint set: 2017-06-12T16:17:12


In [4]:
# Packages

include("/data/notebook_home/Production/Lib/Include-Package-v1.0.jl")
include("/data/notebook_home/Production/Bodies/Individual-Streamline-Body-v2.0.jl")

#timeVariables(2017,5,9,16,0,2017,5,9,16,59)
weeklyTimeVariables(days=7)
#yesterdayTimeVariables()
;

June 4, 2017 23:59:00 to June 11, 2017 23:59:00 Local Time
June 5, 2017 03:59:00 to June 12, 2017 03:59:00 UTC Time


In [5]:
# This is the Mobile Only, Time Based report

UP = UrlParamsInit()
UP.beaconTable = table
UP.resourceTable = tableRt
UP.pageGroup = "%"
UP.urlRegEx = "%"
UP.urlFull = "%"
#UP.limitRows = 3
UP.limitRows = 250
UP.samplesMin = 10
UP.sizeMin = 10000
UP.usePageLoad=false
UP.timeLowerMs = 2000.0
UP.timeUpperMs = 60000.0
UP.orderBy = "time"
UP.deviceType = "mobile"

SP = ShowParamsInit()
SP.desktop=false
SP.mobile=true
SP.devView=false
SP.criticalPathOnly=true
SP.debug=false
SP.debugLevel = 0   # Tests use even numbers with > tests, make this an odd number or zero

LV = LocalVars(false) #useJSon = false
;

In [6]:
urlListDF = returnMatchingUrlTableV2(UP,tv.startTimeMsUTC,tv.endTimeMsUTC)
if (SP.debugLevel > 4)
    beautifyDF(urlListDF[1:min(3,end),:])
end

newListDF = urlListDF[Bool[x > UP.samplesMin for x in urlListDF[:cnt]],:]
topUrlList = newListDF[:urlgroup]
topUrls = cleanupTopUrlTable(topUrlList)
if (SP.debugLevel > 8)
    println(topUrls)
end
;

In [7]:
finalUrlTableOutput(UP,SP,topUrls,tv.startTimeMs,tv.endTimeMs)
;

Recent Urls (Sorted By Time Descending; Min Samples 10 ; Top 3 Page Views),Time,Request Made,Page Size,Samples
/news.nationalgeographic.com/2017/06/wildlife-watch-china-donkey-tigers-zoo/%,"18,225",79,"813,745","33,800"
/news.nationalgeographic.com/2017/06/spider-wasp-huntsman-australia-ibis-video/%,"17,897",98,"839,910","10,603"
/news.nationalgeographic.com/2017/06/baby-bird-dinosaur-burmese-amber-fossil/%,"16,950",95,"972,855","45,209"
